In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# from pandas_profiling import ProfileReport

# Load scripts from parent path
import sys, os
sys.path.insert(0, os.path.abspath('..'))

In [2]:
from scripts.processing import load_train_data, process_data, add_week_month_info

train_raw = load_train_data()
# train = process_data(train_raw)
train = add_week_month_info(train_raw)

train_raw.head()

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,week,month
0,2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,a,1.0,1,1
1,2013-01-01,379.0,2.0,0.0,0.0,0.0,0.0,a,1.0,1,1
2,2013-01-01,378.0,2.0,0.0,0.0,0.0,0.0,a,1.0,1,1
3,2013-01-01,377.0,2.0,0.0,0.0,0.0,0.0,a,1.0,1,1
4,2013-01-01,376.0,2.0,0.0,0.0,0.0,0.0,a,1.0,1,1


### Calculate monthly store sales numbers

In [3]:
store_sales = train_raw.groupby(by=['Store', 'month']).agg(
    Store_Sales_mean=('Sales', 'mean'),
    Store_Customers_mean=('Customers', 'mean')
    )
store_sales = store_sales.reset_index()
store_sales

,Store,month,Store_Sales_mean,Store_Customers_mean
0,1.0,1,3926.966102,487.442623
1,1.0,2,4175.722222,508.716981
2,1.0,3,4273.250000,517.338983
3,1.0,4,3912.875000,471.948276
4,1.0,5,3784.733333,454.762712
...,...,...,...,...
13375,1115.0,8,4888.290323,351.000000
13376,1115.0,9,4336.900000,313.185185
13377,1115.0,10,4577.866667,326.689655
13378,1115.0,11,5263.888889,347.857143


### Load store details

In [4]:
store_details_raw = pd.read_csv("../data/store.csv")
store_details_raw

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [5]:

# Get promo info
def get_promo_info(store_details_raw):
    col = store_details_raw.loc[:, 'PromoInterval'].str.split(',')
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    promo_info = pd.DataFrame(col.copy(), index=store_details_raw.loc[:, 'Store'])
    for month_id, month in enumerate(months):
        promo_info.loc[:, f"{month_id+1}"] = col.apply(lambda x: (1 if month in x else 0) if type(x) is not float else 0)
    promo_info = promo_info.drop('PromoInterval', axis=1)
    return promo_info

promo_info = get_promo_info(store_details_raw)

## Combine all info and save

False

In [13]:
# Cols to store
cols_to_store = ['Store', 'StoreType', 'Assortment', 'CompetitionDistance']
store_details = store_details_raw.loc[:, cols_to_store]
# Fill missing distances with 0
store_details.loc[:, 'CompetitionDistance'] = store_details.loc[:, 'CompetitionDistance'].fillna(store_details.loc[:, 'CompetitionDistance'].mean())

# Add promo feature
promo_info = get_promo_info(store_details_raw)
# has_promo = store_sales.apply(lambda x: promo_info.loc[int(x['Store']), str(int(x['month']))], axis=1)
# store_details.loc[:, 'has_promo'] 

# Add sales
store_details = pd.merge(store_details, store_sales, how='left', on=['Store'])

store_details.to_csv('../data/store_info.csv', index=False)


store_info = pd.read_csv('../data/store_info.csv')
store_info

,Store,StoreType,Assortment,CompetitionDistance,month,Store_Sales_mean,Store_Customers_mean
0,1,c,a,1270.0,1,3926.966102,487.442623
1,1,c,a,1270.0,2,4175.722222,508.716981
2,1,c,a,1270.0,3,4273.250000,517.338983
3,1,c,a,1270.0,4,3912.875000,471.948276
4,1,c,a,1270.0,5,3784.733333,454.762712
...,...,...,...,...,...,...,...
13375,1115,d,c,5350.0,8,4888.290323,351.000000
13376,1115,d,c,5350.0,9,4336.900000,313.185185
13377,1115,d,c,5350.0,10,4577.866667,326.689655
13378,1115,d,c,5350.0,11,5263.888889,347.857143
